In [6]:
import dotenv

In [7]:
dotenv.load_dotenv()

True

In [8]:
from langchain_community.tools import TavilySearchResults

In [9]:
tavily = TavilySearchResults(max_results=2)

In [10]:
tavily.invoke("Who is the PM of India?")

[{'title': 'Prime Minister of India - Wikipedia',
  'url': 'https://en.wikipedia.org/wiki/Prime_Minister_of_India',
  'content': 'Rao, Atal Bihari Vajpayee, Manmohan Singh, and Narendra Modi, who is the current prime minister of India, serving since 26 May 2014. He is the first non-Congress leader to win three consecutive general elections and secure a third successive term (2014, 2019, 2024). The first prime minister to do so was Jawaharlal Nehru, who won the general elections of 1952, 1957, and 1962. [...] The prime minister of India (ISO: _Bhārata kē Pradhānamantrī_) is the head of government( of the Republic of India. Executive authority is vested in the prime minister and his chosen Council of Ministers,( despite the president of India being the nominal head of the executive.( The prime minister has to be a member of one of the houses of bicameral Parliament of India, alongside heading the respective house.( The prime minister and the cabinet are at all times responsible to the Lo

In [12]:
!pip install bs4

  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Using cached soupsieve-2.7-py3-none-any.whl (36 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [bs4]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [14]:
## Retriever
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama.embeddings import OllamaEmbeddings

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=20)
splitted_web_docs = WebBaseLoader(web_path="https://en.wikipedia.org/wiki/Attention_Is_All_You_Need").load_and_split(text_splitter=splitter)
embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")
store = FAISS.from_documents(splitted_web_docs, embedding=embeddings)
retiriever = store.as_retriever()


In [15]:
retiriever.invoke("What does attention actually mean?")

[Document(id='d6be10c8-ea91-4ea7-912d-eb1805d96b98', metadata={'source': 'https://en.wikipedia.org/wiki/Attention_Is_All_You_Need', 'title': 'Attention Is All You Need - Wikipedia', 'language': 'en'}, page_content='Attention Is All You Need - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\

In [16]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever=retiriever,
    name='attention_retriever',
    description='Helps find info about Attention is all you need research paper.'
)

In [17]:
tools = [
    tavily, retriever_tool,
]

In [20]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [25]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_ollama.chat_models import ChatOllama

llm = ChatOllama(model="llama3.1:8b")
agent = create_tool_calling_agent(llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [27]:
executor.invoke({"input": "What is the meaning of life as per hindu culture?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'meaning of life hindu culture'}`


[{'title': 'Hinduism: A Way Of Life Based On Eternal Truths - Harmony Circle', 'url': 'https://www.harmonycircle.sg/resources/religion/hinduism/', 'content': 'In Hinduism, the purpose of life is to recognise the essential oneness of existence. This, however, is often clouded by the illusion of duality â\x80\x93 the belief that one is separate from others and Brahman â\x80\x93 often reinforced by our experiences in the physical world. [...] Unlike other religions, Hinduism does not have a specific founder. Neither did it originate from a particular period of human history. Rather, it is a combination of philosophies, beliefs and practices that are recorded in ancient scriptures called the Vedas, meaning â\x80\x9cknowledgeâ\x80\x9d. [...] While Hinduism draws from many sources, the central focus of its practice is to free oneself from cycles of reincarnation

{'input': 'What is the meaning of life as per hindu culture?',
 'output': 'According to Hindu culture, the meaning of life is to recognize the essential oneness of existence and achieve enlightenment by freeing oneself from cycles of reincarnation. This can be achieved by following four aims in life: dharma (morality and ethics), kama (love, desire, and pleasure), artha (wealth and prosperity), and moksha (salvation). The ultimate goal is to break free from the cycle of samsara through gaining good karma by doing good actions and deeds.'}

In [28]:
executor.invoke({"input": "Describe the attention is all you need paper."})



> Entering new AgentExecutor chain...

Invoking: `attention_retriever` with `{'query': 'Attention is all you need paper'}`


Attention Is All You Need - Wikipedia




































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages



















Search











Search






















Appearance
















Donate

Create account

Log in








Personal tools





Donate Create account Log in





		Pages for logged out editors learn more



ContributionsTalk




























Contents
move to sidebar
hide




(Top)





1
Authors








2
Methods discussed and introduced








3
Historical context




Toggle Historical context subsection





3.1
Predecessors

"Attention Is All You Need"[1] is a 2017 landmark[2][3] research paper in mac

{'input': 'Describe the attention is all you need paper.',
 'output': 'The "Attention is all you need" paper introduced a new deep learning architecture called the transformer, based on the attention mechanism. This architecture has become foundational in modern artificial intelligence and has contributed to the AI boom. The focus of the research was initially on improving Seq2seq techniques for machine translation, but it also showed potential for other tasks like question answering and multimodal generative models.\n\nKey aspects of the paper include:\n\n* Predecessors: The authors discussed the historical context and predecessors of their work.\n* Attention with seq2seq: They introduced the attention mechanism as a way to improve Seq2seq techniques.\n* Parallelizing attention: The authors showed how to parallelize the attention mechanism for faster computation.\n* AI boom era: The paper\'s introduction of the transformer architecture contributed to the AI boom and is considered a la